<a href="https://colab.research.google.com/github/rishi255/softlab-a3/blob/main/softlab_data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -qU omdb
%pip install -qU torch
!git clone https://github.com/huggingface/transformers

from bs4 import BeautifulSoup
import json
import omdb
import os
import requests
import re
import traceback
import torch

print("All installed and imported!")

# Data Scraping

In [ ]:
# get screenplay of all movies returned in the search
def scrape(name_with_dash_list):
    screenplays = []
    print(name_with_dash_list)
    
    # for movie_id, movie_title in movies_list:
    for movie_name_with_dash in name_with_dash_list: 
        print("For movie", movie_name_with_dash)
        # name_with_dash = "-".join(movie_title.split())
        screenplay_url = f"https://imsdb.com/scripts/{movie_name_with_dash}.html"
        print("Attempting to get the screenplay from url", screenplay_url)
        # XPATH = /html/body/table[2]/tbody/tr/td[3]/table/tbody/tr/td/pre/pre
        response = requests.request(method="get", url=screenplay_url)
        soup = BeautifulSoup(response.text, "html.parser")
        try:
            screenplay_text = soup.find('td', class_="scrtext").pre.pre
            # print("Here", type(screenplay_text), str(screenplay_text)[:300])
            try:
                screenplay_text = screenplay_text.text
            except:
                screenplay_text = ""
        except:
            print(f"Movie {movie_name_with_dash} not found!")
            traceback.print_exc()
            # return None
        else:
            screenplays.append(screenplay_text)
            print(f"*** {movie_name_with_dash} done!")    

    with open("merged.txt", "w+") as f:
        f.writelines(screenplays)

scrape(["Jurassic-Park", "Jurassic-Park-III", "Jurassic-Park-The-Lost-World"])

['Jurassic-Park', 'Jurassic-Park-III', 'Jurassic-Park-The-Lost-World']
For movie Jurassic-Park
Attempting to get the screenplay from url https://imsdb.com/scripts/Jurassic-Park.html
*** Jurassic-Park done!
For movie Jurassic-Park-III
Attempting to get the screenplay from url https://imsdb.com/scripts/Jurassic-Park-III.html
*** Jurassic-Park-III done!
For movie Jurassic-Park-The-Lost-World
Attempting to get the screenplay from url https://imsdb.com/scripts/Jurassic-Park-The-Lost-World.html
*** Jurassic-Park-The-Lost-World done!


# Data Cleanup

In [ ]:
# !cat JP1.txt JP2.txt JP3.txt > merged.txt
# !wc -l JP1.txt
# !wc -l JP2.txt
# !wc -l JP3.txt

!wc -l screenplays.txt
# !wc -c merged.txt

# Actual Training and Predicting stuff



In [ ]:
%pip install transformers/
%pip install -r transformers/examples/language-modeling/requirements.txt

In [ ]:
!ls
!pwd
!python transformers/examples/language-modeling/run_clm.py \
    --output_dir=output \
    --model_type=gpt2 \
    --model_name_or_path=gpt2-medium \
    --do_train \
    --train_file=/content/merged.txt \
    --overwrite_output_dir\
    --block_size=200\
    --per_device_train_batch_size=1\
    --save_steps 5000\
    --num_train_epochs=2

merged.txt  sample_data  transformers
/content
2021-04-12 07:31:17.695001: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
04/12/2021 07:31:20 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
04/12/2021 07:31:20 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=output, overwrite_output_dir=True, do_train=True, do_eval=None, do_predict=False, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=2.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs/Apr12_07-31-20_f40b5459fdea, logging_strategy=Interva

In [ ]:
import subprocess
user_prompt = input("Enter the prompt for which the screenplay is to be generated: ")

output = subprocess.Popen(f"python transformers/examples/text-generation/run_generation.py "
    "--model_type gpt2 "
    "--model_name_or_path output "
    "--length 1000 "
    f"--prompt \"{user_prompt}\" "
    "> test.txt", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
std_out, std_err = output.communicate()

print("OUTPUT:", std_out.decode())
print("ERROR:", std_err.decode())

Enter the prompt for which the screenplay is to be generated: The T-rex let out a huge roar
OUTPUT: 
ERROR: 2021-04-12 07:48:39.733239: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
04/12/2021 07:48:41 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
04/12/2021 07:48:58 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=1000, model_name_or_path='output', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='', prompt='The T-rex let out a huge roar', repetition_penalty=1.0, seed=42, stop_token=None, temperature=1.0, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



In [ ]:

names = ["Alan", "Grant", "Ellie", "Ian", "John", "Hammond", "Robert","Muldoon", "Donald", "Gennaro",
 "Henry", "Timmy", "Lex", "Arnold", "Ray", "Dennis", "Nedry", "Harding", "Lewis","Dodgson",
  "Mr. DNA", "Ed", "Regis", "Marty", "Sarah", "Harding", "Roland","Tembo", "Peter", "Ludlow", "Nick", 
  "Kelly", "Curtis","Dieter", "Stark","Ajay","Eddie","Robert","Richard",
  "Jack","Arby", "Benton", "Howard", "George", "Baselton","Diego", "Paul", "Kirby", "Amanda",
   "Billy", "Brennan", "Eric", "Kirby", "Udesky", "Cooper", "Nash", "Mark", "Ben", "Hildebrand", "Enrique", "Cardoso", "Charlie"]

names = [i.upper() for i in names]
print(names)

with open("test.txt") as f1, open("formatted.txt", "w") as f2:
    lines = f1.readlines()
    newlines = [] 
    for i in lines:
        # print(re.sub(r"\W([A-Z]{2,})\W", r"\1:\n", i))
        a = re.sub(r"(" + r"|".join(names) + r")", r"\n\1", i)
        # a = re.sub(r"GENERATED", r"\nBRUH: ", i)
        newlines.append(a)
        # print("Found in", a)

    # print(lines)
    f2.writelines(newlines)

['ALAN', 'GRANT', 'ELLIE', 'IAN', 'JOHN', 'HAMMOND', 'ROBERT', 'MULDOON', 'DONALD', 'GENNARO', 'HENRY', 'TIMMY', 'LEX', 'ARNOLD', 'RAY', 'DENNIS', 'NEDRY', 'HARDING', 'LEWIS', 'DODGSON', 'MR. DNA', 'ED', 'REGIS', 'MARTY', 'SARAH', 'HARDING', 'ROLAND', 'TEMBO', 'PETER', 'LUDLOW', 'NICK', 'KELLY', 'CURTIS', 'DIETER', 'STARK', 'AJAY', 'EDDIE', 'ROBERT', 'RICHARD', 'JACK', 'ARBY', 'BENTON', 'HOWARD', 'GEORGE', 'BASELTON', 'DIEGO', 'PAUL', 'KIRBY', 'AMANDA', 'BILLY', 'BRENNAN', 'ERIC', 'KIRBY', 'UDESKY', 'COOPER', 'NASH', 'MARK', 'BEN', 'HILDEBRAND', 'ENRIQUE', 'CARDOSO', 'CHARLIE']
